In [3]:


import sys
import serial
import pyqtgraph as pg
from PyQt5.QtWidgets import QApplication, QMainWindow
from PyQt5.QtCore import QThread, pyqtSignal, QTimer

class SerialReader(QThread):
    data_received = pyqtSignal(str)

    def __init__(self, serial_port):
        super().__init__()
        self.serial_port = serial_port
        self.running = True

    def run(self):
        while self.running:
            try:
                line = self.serial_port.readline().decode().strip()
                if line:
                    self.data_received.emit(line)
            except serial.SerialException as e:
                print(f"Serial error: {e}")
                self.running = False

    def stop(self):
        self.running = False
        self.quit()
        self.wait()
import time
from PyQt5.QtCore import QThread

class DataSender(QThread):
    def __init__(self, serial_port, file_path):
        super().__init__()
        self.serial_port = serial_port
        self.file_path = file_path
        self.running = True
        self.last_send_time = time.perf_counter()  # Start the timer

    def run(self):
        with open(self.file_path, 'r') as file:
            self.file = file
            interval = 0.0005  # 0.5 milliseconds
            while self.running:
                current_time = time.perf_counter()
                elapsed_time = current_time - self.last_send_time
                
                if elapsed_time >= interval:
                    line = self.file.readline()
                    if not line:
                        self.file.seek(0)  # Loop back to the start of the file
                        line = self.file.readline()
                    self.serial_port.write(line.encode())
                    self.last_send_time = current_time  # Reset the timer

    def stop(self):
        self.running = False
        self.quit()
        self.wait()


class ECGPlotter(QMainWindow):
    def __init__(self, serial_port, *args, **kwargs):
        super(ECGPlotter, self).__init__(*args, **kwargs)
        self.setWindowTitle('ECG Signal Plotter')
        self.setGeometry(100, 100, 800, 600)

        # Create a plot widget
        self.plotWidget = pg.PlotWidget()
        self.setCentralWidget(self.plotWidget)

        # Initialize the data
        self.data = []
        self.buffer_size = 100  # Maximum number of samples to display at a time

        # Set up the plot
        self.plotWidget.setLabel('left', 'ECG Signal')
        self.plotWidget.setLabel('bottom', 'Sample Index')
        self.plotWidget.showGrid(True, True, 0.5)
        self.curve = self.plotWidget.plot(pen='r')

        # Use the passed serial port
        self.ser = serial_port

        # Initialize and start the SerialReader thread
        self.serial_reader = SerialReader(self.ser)
        self.serial_reader.data_received.connect(self.updatePlot)
        self.serial_reader.start()

        # Initialize and start the DataSender thread
        self.data_sender = DataSender(self.ser, 'C:/Users/ARTAPC/Desktop/Git/test/src/ecg_with_peaks_downsampled.txt')
        self.data_sender.start()

        self.peak_lines = []  # To keep track of the peak lines

    def updatePlot(self, line):
        try:
            read_sample, read_is_peak = line.split(',')
            read_sample = float(read_sample)
            read_is_peak = int(read_is_peak)

            # Append data to buffer
            self.data.append(read_sample)
            if len(self.data) > self.buffer_size:
                self.data.pop(0)  # Remove the oldest sample to maintain buffer size

            # Update the plot with the current data
            self.curve.setData(self.data)

            # Update peak markers with vertical lines
            # If a peak is detected, add it at the correct position within the buffer
            if read_is_peak == 1:
                peak_position = len(self.data) - 1  # Position within the buffer
                peak_line = pg.InfiniteLine(pos=peak_position, angle=90, pen=pg.mkPen(color='g', width=2))
                self.plotWidget.addItem(peak_line)
                self.peak_lines.append(peak_line)

            # Remove old peak lines that fall out of the buffer
            if len(self.peak_lines) > 0:
                for peak_line in self.peak_lines:
                    if peak_line.pos().x() < 0:
                        self.plotWidget.removeItem(peak_line)
                # Filter out the removed peak lines from the list
                self.peak_lines = [line for line in self.peak_lines if line.pos().x() >= 0]

            # Reposition the peak lines relative to the buffer
            for peak_line in self.peak_lines:
                peak_line.setPos(peak_line.pos().x() - 1)

        except ValueError:
            pass  # Handle any parsing errors

    def closeEvent(self, event):
        """Override closeEvent to properly close serial port and stop threads."""
        print("Window is closing")
        self.serial_reader.stop()  # Stop the serial reader thread
        self.data_sender.stop()  # Stop the data sender thread
        if self.ser.is_open:
            self.ser.close()  # Close the serial port immediately
            print("Serial port closed")
        event.accept()  # Accept the event to close the window

# Initialize the serial port and run the application
serial_port = serial.Serial('COM11', baudrate=38400, timeout=None, parity=serial.PARITY_EVEN, rtscts=False, xonxoff=True, dsrdtr=False)

app = QApplication(sys.argv)
window = ECGPlotter(serial_port)
window.show()
sys.exit(app.exec_())


Window is closing
Serial port closed


SystemExit: 0

c:\Users\ARTAPC\miniconda3\envs\py38\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
